In [1]:
%config Completer.use_jedi = False

# Setup notebook 

## If running on Google Colab

In [2]:
# # Mount Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# # Install libraries
# %%capture
# !pip install datasets transformers striprtf

## Import libraries

In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import torch
from os import listdir
from os.path import isfile, join
from striprtf.striprtf import rtf_to_text
import importlib
deep_translator = importlib.import_module("deep_translator")
from deep_translator import GoogleTranslator

# Functions

In [4]:
def get_answers(my_context, my_questions):
    my_answers = []
    for q in my_questions:
        QA_input = {
            'question': q,
            'context': my_context
        }
        res = nlp(QA_input)
        my_answers.append(res)
    return my_answers

In [5]:
def print_answers(my_context, my_questions, my_answers):
    print('*'*50+' CONTEXT '+'*'*50)
    print(my_context)
    for i, (q, a) in enumerate(zip(my_questions, my_answers)):
        print(f'********** QUESTION {i+1} **********')
        print(f'"{q}"')
        print(f'ANSWER = {a["answer"]}')
        print(f'SCORE = {"{:.2f}".format(a["score"]*100)}%\n')

In [6]:
def translate_context(context, source_language, target_language):
    if len(context)>5000:
        new_context = 'Context too long, I will not translate it!'
    else:
        new_context = GoogleTranslator(source=source_language, target=target_language).translate(context)
    return new_context

# Load medical document

In [7]:
# # Specify base_path
# # base_path = '/content/drive/MyDrive/HealthBigData/Data/From_Giuliani_Francesco'
# # base_path = '/content/drive/MyDrive/HealthBigData/Data/From_Boveri_Sara'
# # Get paths of all files in base_path
# file_names = [base_path+'/'+f for f in listdir(base_path) if isfile(join(base_path, f))]
# #DEBUGONLY
# print(file_names[0])

# Indexes of specific condition contexts
# Short context: 0

# # Get content of specific document
# with open(file_names[0]) as infile:
#     content = infile.read()
#     my_context = rtf_to_text(content)
# print(my_context)

# Get answers - italian

In [8]:
# Model from HuggingFace
model_checkpoint = 'mrm8488/bert-italian-finedtuned-squadv1-it-alfa'

In [9]:
# Made-up context, short
short_context = '''
Paziente di 70 anni, uomo. Esame eseguito d'urgenza, in condizioni basali.
Nella profondità della sostanza bianca in sede temporo-parietale dx si apprezza fornazione
grossolanamente rotondeggiante di circa 2 cm circondata da imponente reazione edemigena.
Altra più minuta area ipodensa della sostanza bianca sembra apprezzarsi in sede frontale sin.
Al momento attuale non evidenza di spandimenti emorragici intracranici.
Strutture mediane in asse.
Complesso ventricolare di normale ampiezza e morfologia.
Opportuno potrebbe essere approfondimento diagnostico a conferma eventualmente con esame RM
Pertanto si dimette il pz con i seguenti consigli:
- Keppra 500 1 cpr per 2 volte al dì;
- Soldesam 4 mg 1 fl im al dì  im per 15 giorni; successivamente deltacortene 25 mg 1 cpr per 2 volte al dì per 5 giorni ; successivamente 1 cpr al dì per ulteriori 5 giorni;
- Antra da 20 mg 1 cpr / die per 20 giorni;
- RMN encefalo fra 30 giorni e successiva rivalutazione neurologica per decidere ulteriore ed evntuale piano di trattamento. 
'''

In [10]:
# Made-up context, long
long_context = '''Gentile collega, si dimette il suo assistito in data: 16.06.2015

con la seguente diagnosi: 
INSUFFICIENZA CARDIACA DIASTOLICA CON DILATAZIONE DELLE SEZIONI DX IN FA IN TAO IN LABILE COMPENSO (SCOMPENSO CONGESTIZIO= EDEMI DEGLI ARTI INFERIORI COSPICUI)) CON  IPERTENSIONE POLMONARE. ULCERE DA FLEBOSTASI DEGLI ARTI INFERIORI INFETTE (  PSEUDOMONAS AERUGINOSA ) . GONARTROSI SEVERA DX SOTTOPOSTA AD ARTROCENTESI. DIABETE MELLITO TIPO 2 SCOMPENSATO E COMPLICATO DA NEFROPATIA.  INSUFFICIENZA RENALE CRONICA  REGREDITA.
 
Notizie anamnestiche: 
Paziente non tabagista, affetta da ipertensione arteriosa, diabete mellito tipo 2, fibrillazione atriale permanente in NAO. Nega allergie. A seguito di comparsa di iperpiressia poco responsiva a politerapia antibiotica (claritromicina, piperacillina/tazobactam) e dolore progressivo a livello del ginocchio dx limitante la deambulazione, giunge in PS ove esegue visita radiogramma del ginocchio dx e visita ortopedica che, in ragione di una sospetta artrite settica, consiglia il ricovero in ambiente internistico per le valutazioni e le cure del caso. 

Esami ematochimici alterati alla dimissione:
Hb glicosilata 8,2; GFR: 85ml/min; Creatinina:0.7mg; Azotemia: 23.0mg,  pro-BNP: 3139.0; Microalbuminuria 60,5; Sodio 132 --> 135; glicemia 210 --> 41; Sideremia 42; Fosfatasi alcalina 110; LDH 233; Hb 12,5; pro-BNP 5103; Albumina 38,9; Alfa-1-globuline 7,1; Alfa-2-globuline 14,8; Beta-2-globuline 6,7; Gamma-globuline 44,3; VES 52; PCR: 5.18, es.tampone ulcera cutanea dx:Positivo per Pseudomonas sensibile a Ciprofloxacina; es. liquido biologico: cellule 35200, Cell. prevalenza di  polimormonucleati,,LDH: 1330.0, Prot.Tot: 5.00, es. colturale : neg. 
   

Esami strumentali: 
RX DEL TORACE - RX DEL GINOCCHIO DX (4/06/2015): Marcati fenomeni gonartrosici tricompartimentali a dx. Marcata tumefazione dei tessuti molli del ginocchio dx, particolarmente evidente in sede sovrarotulea come per presenza di versamento nel recesso sinoviale sottoquadricipitale. Diffuse irregolari calcificazioni vascolari femoro-popliteo-tibio-peroneali. Osteopenia. L'esame del torace non evidenzia consolidazioni polmonari in atto di significato flogistico né falde di versamento pleurico. Immagine cardiaca ingrandita. Segni di stasi del piccolo circolo. Aorta ectasica e calcifica.
ELETTROCARDIOGRAMMA (5/06/2015): Fibrillazione atriale a 82 bpm; BBDx quasi completo. Anomalie in parte secondarie della ripolarizzazione ventricolare. 
VISITA REUMATOLOGICA (7/06/2015): Circa 8 giorni fa' insorgenza di febbre e gonartralgia dx con tumefazione articolare. Non dolore articolare a riposo. Non febbre attualmente. EO: segni di versamento articolare del ginocchio dx, non rubor, limitazione funzionale articolazione ginocchio dx, lieve edema della gamba dx. Recente infezione ulcera gamba dx. In terapia anticoagulante. Conclusione: probabile emartro ginocchio dx in gonartrosi severa. Si consiglia: borsa di ghiaccio per 20 minuti per 2 vv/die. Riposo arti inferiori. Monitorare PCR, elettroforesi proteica ogni 5 giorni. ETG ginocchio. 
ECOCARDIOGRAMMA (10/06/2015): Esame eseguito a letto della paziente, poco collaborante. Con tali limiti si documenta: Ritmo da F.A. Ventricolo sinistro lievemente ipertrofico (spessore setto 11,4 mm), non dilatato, con funzione contrattile globale ai limiti inferiroi di norma (EF 50% circa). Dilatazioni bi-atriale, maggiore a Dx. Lieve-medio rigurgito mitralico. Lieve rigurgito aortico, in valvola sclerotica. Ventricolo Dx dilatato e lievemente ipocinetico (TAPSE 17 mm). Moderato-severa IT con PAPs stimata 25 mmHg + 15 mmHg. VCI dilatata e ipocollassante (27/20 mm). 

Osservazioni / Terapie effettuate: 
Nel corso della degenza la paziente è stata sottoposta ad artrocentesi ( 17.0718)  del ginocchio dx con prelievo di liquido sinoviale; sottoposta a terapia antibiotica empirica prima e poi mirata ( ATB) si è assistito a regressione della febbre e miglioramento clinico e delle ulcere cutanee degli arti inf.; inoltre sottoposta a t. diuretica ev  gli edemi degli arti inferiori sono quasi regrediti.  

Trattamento a domicilio e indicazioni: 
DIETA IPOSODICA E IPOCALORICA 
    • Eucreas 50 mg/1000 mg 1 cp per 2 vv/die
    • Baypress 20 mg - 1 cp/die
    • Eliquis 2,5 1 cp per 2 vv/die
    • Carvedilolo 6,25 mg - 1 cp/die
    • Lasix 25 mg - 1 cp per 2 vv/die
    • Luvion 50 mg - 1 cp/die 
    • Ciproxin 500 mg - 1 cp per 2 vv/die per 10 giorni
    • Medicazione cute arto inf. dx con fitostimoline 


Esegua inoltre:
    • Funzionalità renale, elettroforesi proteica, PCR tra 5-7 giorni circa
    • Cons. Reumatologica +ETG ginocchio dx tra 15 giorni 

Controlli periodicamente la pressione arteriosa, il peso corporeo, gli esami di funzionalità renale, epatica, l'emocromo.
'''

In [11]:
my_questions = [
    'Qual è la condizione patologica?',
    'Qual è l\'età?',
    'Qual è il sesso?',
    'Quali farmaci assume attualmente?',
    'Quali sono le procedure chirurgiche applicate?',
]

In [12]:
nlp = pipeline('question-answering', model=model_checkpoint, tokenizer=model_checkpoint)

In [13]:
# Define my_context
my_context = short_context

In [14]:
my_answers = get_answers(my_context, my_questions)
print_answers(my_context, my_questions, my_answers)

**************************************************CONTEXT**************************************************

Paziente di 70 anni, uomo. Esame eseguito d'urgenza, in condizioni basali.
Nella profondità della sostanza bianca in sede temporo-parietale dx si apprezza fornazione
grossolanamente rotondeggiante di circa 2 cm circondata da imponente reazione edemigena.
Altra più minuta area ipodensa della sostanza bianca sembra apprezzarsi in sede frontale sin.
Al momento attuale non evidenza di spandimenti emorragici intracranici.
Strutture mediane in asse.
Complesso ventricolare di normale ampiezza e morfologia.
Opportuno potrebbe essere approfondimento diagnostico a conferma eventualmente con esame RM
Pertanto si dimette il pz con i seguenti consigli:
- Keppra 500 1 cpr per 2 volte al dì;
- Soldesam 4 mg 1 fl im al dì  im per 15 giorni; successivamente deltacortene 25 mg 1 cpr per 2 volte al dì per 5 giorni ; successivamente 1 cpr al dì per ulteriori 5 giorni;
- Antra da 20 mg 1 cpr / die p

# Get answers - English

In [15]:
# Model from HuggingFace
model_checkpoint = 'deepset/roberta-base-squad2' # SQUAD2 multilangual model

In [16]:
# Translate context ITA-->ENG
eng_context = translate_context(my_context, source_language='it', target_language='en')

In [17]:
eng_questions = []
for q in my_questions:
    eng_q = translate_context(q, source_language='it', target_language='en')
    eng_questions.append(eng_q)

In [18]:
nlp = pipeline('question-answering', model=model_checkpoint, tokenizer=model_checkpoint)

In [19]:
eng_answers = get_answers(eng_context, eng_questions)

In [20]:
# Translate answers ENG-->ITA
ita_answers = []
for a in eng_answers:
    try:
        a_text = translate_context(a['answer'], source_language='en', target_language='it')
    except: # Some texts will not be translated, for example numbers
        a_text = a['answer']
    a_dict = {
        'score': a['score'],
        'answer': a_text
    }
    ita_answers.append(a_dict)

In [21]:
print_answers(my_context, my_questions, ita_answers)

**************************************************CONTEXT**************************************************

Paziente di 70 anni, uomo. Esame eseguito d'urgenza, in condizioni basali.
Nella profondità della sostanza bianca in sede temporo-parietale dx si apprezza fornazione
grossolanamente rotondeggiante di circa 2 cm circondata da imponente reazione edemigena.
Altra più minuta area ipodensa della sostanza bianca sembra apprezzarsi in sede frontale sin.
Al momento attuale non evidenza di spandimenti emorragici intracranici.
Strutture mediane in asse.
Complesso ventricolare di normale ampiezza e morfologia.
Opportuno potrebbe essere approfondimento diagnostico a conferma eventualmente con esame RM
Pertanto si dimette il pz con i seguenti consigli:
- Keppra 500 1 cpr per 2 volte al dì;
- Soldesam 4 mg 1 fl im al dì  im per 15 giorni; successivamente deltacortene 25 mg 1 cpr per 2 volte al dì per 5 giorni ; successivamente 1 cpr al dì per ulteriori 5 giorni;
- Antra da 20 mg 1 cpr / die p